# Домашнее задание № 5

**Ситуация:**\
Спокойное рабочее утро. \
Налив чашку утреннего кофе вы замечаете в Outlook письмо, в котором коллеги просят помощи в решении проблемы. 

Для тестирования API нового сервиса валидации адресов им необходимо получить списки реальных адресов и геокоординат которыми пользуются клиенты компании.\
Они уже обратились за помощью к коллегам DevOps. Те выгрузили из логов сетевого балансера все, что клиенты передавали в поля, предназначенные для адресов.\
Файл durty_data.csv прилагается к письму.

От вас требуется выделить из файла только похожие на адреса и гео-координаты строки. Убрать дубликаты и кавычки, а также разложить результаты на 2 разных листа Excel файла: строковые адреса на один, а гео-координаты на другой.

Используйте любые полученные в предыдущих уроках навыки для решения поставленной задачи.
- работу с Python
- работу с Unix
- работу с БД

*В идеале оформить решение в виде .ipynb файла!* В нем **должно быть**:
1) описание алгоритма решения поставленной задачи (отдельно в начале или пошагово комментариями к коду);
2) все использованные в ходе решения скрипты/команды/запросы.

Результатом работы по вашему алгоритму должен стать xlsx файл, в котором на разных листах лежат все распознанные адреса и геокоординаты.

***Важно!***
* алгоритм решения должен быть описан настолько просто, чтобы не возникало сложностей с его воспроизведением
* использовать можно только механики/модули/методики, изученные на занятиях


### *Решение*

#### Алгоритм обработки текстовых данных

1. **Чтение данных**: В зависимости от флага `pandas_read_csv` данные считываются либо через `pd.read_csv()`, либо обычным способом через `open()`.  
2. **Декодирование текста**: Функция `decode_unknown()` пытается перекодировать текст из разных кодировок (UTF-8, KOI8-R, CP866 и др.), проверяя, соответствует ли результат критерию русскоязычности (не менее 70% русских символов).  
3. **Очистка данных**: Удаляются лишние пробелы, кавычки, дубликаты, а также символы `\` и `"`.  
4. **Разделение на координаты и адреса**: С помощью регулярных выражений текст разделяется на координаты (формат `числа.число, число.число`) и адреса.  
5. **Фильтрация русскоязычных адресов**: Отбрасываются строки, содержащие менее 3 русских букв.  
6. **Сохранение результатов**: Данные записываются в Excel-файл на разные листы: "Адреса" и "Координаты".  

Итог: код преобразует "грязные" данные в читаемые русскоязычные адреса и координаты, сохраняя их в структурированном виде.

In [1]:
import re
import pandas as pd


__import__("warnings").filterwarnings('ignore')  # Отключаем предупреждения


CONFINES = 0.7  # Порог для распознавания русского текста
EN_LETTER = re.compile(r'[a-z/<>\n\t]', flags=re.IGNORECASE)  # английские буквы
RUS_DIGITS = re.compile(r'[а-яё0-9/., -]', flags=re.IGNORECASE)  # русские буквы и цифры
NOT_PATTERN = re.compile(r'[^а-яё]', flags=re.IGNORECASE)  # кроме русских букв
BYTE_PATTERN = re.compile(r'[a-wyzG-Z .,/<>-]')  # английские буквы, которых нет в байт-строке


def decode_unknown(text):
    """
    Функция перекодировки из кразозябр в читаемый русский текст
    :param text: текст
    :return: перекодированный текст
    """
    # Если в строке более порога русских букв и цифр - то текст уже годный к использованию
    if len(RUS_DIGITS.findall(text)) > len(text) * CONFINES:
        return text

    # Список кодировок для перебора
    encodings = ['UTF-8', 'KOI8-R', 'CP866', 'WINDOWS-1251', 'WINDOWS-1252', ]
    for encoding in encodings:
        try:
            if any(0 <= ord(c) < 255 for c in BYTE_PATTERN.sub('', text)):
                # Если есть хоть один символ от байт-кодов -->
                # Восстанавливаем байт-строку двойной перекодировкой
                text_ = text.encode('latin1').decode('unicode_escape')
                decoded = text_.encode('latin1').decode(encoding)
            else:
                # Обычную строку с кракозабрами пытаемся закодировать и раскодировать в UTF-8
                decoded = text.encode(encoding, errors='ignore').decode('UTF-8')

            # В некоторых кодировках преобладают буквы п, я - уберем их
            temp = EN_LETTER.sub('', decoded).replace('п', '').replace('я', '')
            #  Если в строке русских букв и цифр больше порога - текст годный к использованию
            if len(RUS_DIGITS.findall(temp)) > len(temp) * CONFINES:
                return decoded
        except:
            pass
    # Не смогли распознать кракозябры --> возвращаем как есть и удалим их потом
    return text

In [2]:
def main_func(file_name='durty_data.csv', pandas_read_csv=True, use_decode_text=True):
    """
    Чтение текстового файла, очистка от кавычек, декодирование кракозябр и сохранение в эксель
    :param file_name: Имя текстового файла
    :param pandas_read_csv: Использовать pandas для чтения файла.
    :param use_decode_text: Декодировать кракозябры или оставить как есть.
    :return: 2 датафрейма с найденными адресами и координатами
    """
    pd_suffix = '_pandas'  # Добавляется к имени файла-результата

    if pandas_read_csv:
        print(f'Читаем файл "{file_name}" методом pd.read_csv()')
        # Читаем файл пандасом и переименовываем колонку
        df = pd.read_csv(file_name, encoding='utf-8', header=None).rename(columns={0: 'text'})
    else:
        pd_suffix = ''
        print(f'Читаем файл "{file_name}" обычным методом')
        # Читаем файл обычным методом
        with open(file_name, encoding='utf-8') as file:
            rows = file.readlines()

        # Создаем ДФ
        df = pd.DataFrame(data=rows, columns=['text'])

    df['text_old'] = df['text']  # Для отладки оставим оригинальный текст

    print('len(df):', len(df))

    # Замена "\\" на "\"
    df['text'] = df['text'].str.replace('\\\\', '\\')

    if use_decode_text:
        pd_suffix = f'{pd_suffix}_decoded'  # Добавим еще один суффикс к имени файла
        # Перекодируем кракозябры и удалим пробелы и слэши с концов строки в цикле,
        # т.к. есть двойные перекодировки
        for _ in range(2):
            df['text'] = df['text'].map(decode_unknown).str.strip().str.strip('\\')

    # Очистим строку от кавычек и лишних пробелов
    df['text'] = df['text'].str.strip().str.strip('"').str.strip()

    # Удаляем дубликаты
    df.drop_duplicates(inplace=True)

    print('Уникальных строк в df:', len(df))

    df['len_str'] = df['text'].map(len)  # длина строки для отладки

    # Создаем маску для координат: цифры точка цифры разделитель цифры точка цифры
    coord_mask = df['text'].str.contains(r'^\d+\.\d*[, ]{1,3}\d+\.\d*$', regex=True)
    # Фильтруем координаты
    df_coords = df[coord_mask]
    # Фильтруем адреса
    df_adress = df[~coord_mask]

    # Создаем маску, что в адресе есть русские буквы в количестве не менее 3
    rus_mask = df_adress['text'].map(lambda z: len(NOT_PATTERN.sub('', z)) >= 3
                                     # and '</search>' not in z  # это раскоментарить,
                                     # если нужно выкинуть адреса с тегами
                                     )
    print(f'Не распознано адресов: {len(df_adress) - sum(rus_mask)} из {len(df_adress)}')

    # Оставляем только нужные строки с адресами
    df_adress = df_adress[rus_mask]

    print('Уникальных адресов:   ', len(df_adress))
    print('Уникальных координат: ', len(df_coords))

    # Запись в Excel на разные листы выбранных колонок из списка без заголовков
    # cols = df_coords.columns
    cols = ['text']
    with pd.ExcelWriter(f'result{pd_suffix}.xlsx', engine='xlsxwriter') as writer:
        df_adress[cols].to_excel(writer, sheet_name='Адреса', index=False, header=None)
        df_coords[cols].to_excel(writer, sheet_name='Координаты', index=False, header=None)

    return df_adress, df_coords

#### Без распознавания неверных кодировок

In [3]:
%%time
df1, df2 = main_func(use_decode_text=False)

Читаем файл "durty_data.csv" методом pd.read_csv()
len(df): 207209
Уникальных строк в df: 87452
Не распознано адресов: 15965 из 25078
Уникальных адресов:    9113
Уникальных координат:  62374
CPU times: total: 1.33 s
Wall time: 1.32 s


In [4]:
df1.head(10)

,text,text_old,len_str
2,"Свердловская обл, г Екатеринбург, ул Машинная,...","Свердловская обл, г Екатеринбург, ул Машинная,...",57
3,"Ростовская обл, г Ростов-на-Дону, ул 2-я Волод...","Ростовская обл, г Ростов-на-Дону, ул 2-я Волод...",58
28,"Свердловская обл, г Екатеринбург, ул Машинная,...","Свердловская обл, г Екатеринбург, ул Машинная,...",63
35,"Свердловская обл, г Екатеринбург, ул Машинная,...","Свердловская обл, г Екатеринбург, ул Машинная,...",61
48,"Свердловская обл, г Екатеринбург, ул Краснолес...","Свердловская обл, г Екатеринбург, ул Краснолес...",59
52,"Ханты-Мансийский Автономный округ - Югра АО, г...","Ханты-Мансийский Автономный округ - Югра АО, г...",73
53,"г Санкт-Петербург, Приморское ш, д 455 к 29.","г Санкт-Петербург, Приморское ш, д 455 к 29.",45
96,"Пензенская обл, г Пенза, Симферопольский 7-й п...","Пензенская обл, г Пенза, Симферопольский 7-й п...",54
97,"Краснодарский край, г Сочи, пгт Красная Поляна...","Краснодарский край, г Сочи, пгт Красная Поляна...",58
106,"Респ Крым, г Евпатория, ул Симферопольская д57","Респ Крым, г Евпатория, ул Симферопольская д57",46


#### Полная обработка текста

In [5]:
%%time
df1, df2 = main_func()

Читаем файл "durty_data.csv" методом pd.read_csv()
len(df): 207209
Уникальных строк в df: 87452
Не распознано адресов: 13 из 25078
Уникальных адресов:    25065
Уникальных координат:  62374
CPU times: total: 2.81 s
Wall time: 2.82 s


In [6]:
df1.head(10)

,text,text_old,len_str
2,"Свердловская обл, г Екатеринбург, ул Машинная,...","Свердловская обл, г Екатеринбург, ул Машинная,...",57
3,"Ростовская обл, г Ростов-на-Дону, ул 2-я Волод...","Ростовская обл, г Ростов-на-Дону, ул 2-я Волод...",58
8,"РОССИЯ, 630096, Новосибирская область, г Новос...","Ð ÐÐ¡Ð¡ÐÐ¯, 630096, ÐÐ¾Ð²Ð¾ÑÐ¸Ð±Ð¸ÑÑÐºÐ°...",73
10,"РОССИЯ, 450077, Республика Башкортостан, г Уфа...","Ð ÐÐ¡Ð¡ÐÐ¯, 450077, Ð ÐµÑÐ¿ÑÐ±Ð»Ð¸ÐºÐ° ÐÐ...",66
13,"614500, Пермский край, р-н Пермский, д Хмели, ...","614500, ÐÐµÑÐ¼ÑÐºÐ¸Ð¹ ÐºÑÐ°Ð¹, Ñ-Ð½ ÐÐµÑ...",69
15,"454106, Челябинская обл, Челябинск г, Победы п...","454106, Ð§ÐµÐ»ÑÐ±Ð¸Ð½ÑÐºÐ°Ñ Ð¾Ð±Ð», Ð§ÐµÐ»Ñ...",61
28,"Свердловская обл, г Екатеринбург, ул Машинная,...","Свердловская обл, г Екатеринбург, ул Машинная,...",63
35,"Свердловская обл, г Екатеринбург, ул Машинная,...","Свердловская обл, г Екатеринбург, ул Машинная,...",61
38,"628418, Ханты-Мансийский автономный округ - Юг...","628418, Ð¥Ð°Ð½ÑÑ-ÐÐ°Ð½ÑÐ¸Ð¹ÑÐºÐ¸Ð¹ Ð°Ð²Ñ...",78
46,"г. Севастополь, Шелкунова, 1","Ð³. Ð¡ÐµÐ²Ð°ÑÑÐ¾Ð¿Ð¾Ð»Ñ, Ð¨ÐµÐ»ÐºÑÐ½Ð¾Ð²Ð°, 1",28
